In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 11606770
paper_name = 'birrell_brown_2001' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [7]:
original_data = pd.read_csv('raw_data/inline-supplementary-material-1.txt', sep='\t')

In [8]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4297 x 55


In [9]:
original_data.head()

,20040824,NAME,WEIGHT,DESC,AAPO,YPD,Wort,RafA,Lova,HygB,...,SC,Sorb-5g,Sorb-15g,Thr,Tpt,TPZ,Trp,UVA,UVB,UVC
0,YML095C,RAD10,1,removal of nonhomologous ends*,-0.480,0.014,-0.233,-0.137,0.035,0.063,...,-0.210,-0.085,-0.152,-0.211,-0.045,-1.540,-0.171,-0.248,-3.200,-3.082
1,YER162C,RAD4,1,"nucleotide-excision repair, DNA damage recogni...",-0.091,0.047,-0.059,0.052,0.054,0.106,...,-0.220,-0.093,-0.042,-0.201,0.128,-1.301,-0.239,-0.230,-3.296,-3.205
2,YLR032W,RAD5,1,DNA repair,-0.134,-0.544,-0.105,0.047,0.117,-0.257,...,-0.039,0.327,0.388,0.233,-0.046,-1.182,-0.074,-0.332,-2.529,-2.556
3,YMR201C,RAD14,1,"nucleotide-excision repair, DNA damage recogni...",-0.242,0.085,0.005,0.034,-0.078,-0.034,...,-0.234,-0.136,-0.224,-0.401,-0.208,-1.135,-0.220,0.178,-2.912,-2.762
4,YGR258C,RAD2,1,"nucleotide-excision repair, DNA incision, 3'-t...",-0.044,0.016,0.080,-0.060,0.131,0.047,...,-0.115,-0.066,0.007,-0.089,0.087,-0.675,-0.100,-0.098,-3.555,-3.417


In [10]:
original_data['orf'] = original_data['20040824'].astype(str)

In [11]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [12]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [13]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [20040824, NAME, WEIGHT, DESC, AAPO, YPD, Wort, RafA, Lova, HygB, GlyE, Calc, Caff, AntA, ActD, Alk-5g, Alk-15g, AraC, Arn, Ars, Ben, Bleo, Cis1, Cis4, CPTa, CPTc, Dox, Gal-5g, Gal-15g, H2O2, HU, Ida, IR, Lys, Mech, Mel, Min-5g, Min-15g, MMC, MMS, NaCl-5g, NaCl-15g, Nys-5g, Nys-15g, Oxa, SC, Sorb-5g, Sorb-15g, Thr, Tpt, TPZ, Trp, UVA, UVB, UVC, orf]
Index: []

[0 rows x 56 columns]


In [14]:
original_data.set_index('orf', inplace=True)

In [15]:
original_data = original_data[['UVB','UVC']].apply(pd.to_numeric, axis=1, errors='coerce')

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.shape

(4258, 2)

# Prepare the final dataset

In [18]:
data = original_data.copy()

In [19]:
dataset_ids = [745, 466]
datasets = datasets.reindex(index=dataset_ids)

In [20]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [21]:
data.head()

dataset_id,745,466
data_type,value,value
orf,,
YAL002W,-0.013,0.106
YAL004W,-0.049,-0.028
YAL005C,-0.074,-0.055
YAL007C,-0.008,-0.018
YAL008W,-0.034,-0.028


## Subset to the genes currently in SGD

In [22]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [23]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,745,466
,data_type,value,value
gene_id,orf,,
2,YAL002W,-0.013,0.106
1863,YAL004W,-0.049,-0.028
4,YAL005C,-0.074,-0.055
5,YAL007C,-0.008,-0.018
6,YAL008W,-0.034,-0.028


# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,745,466,745,466
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,-0.013,0.106,0.009707,0.600346
1863,YAL004W,-0.049,-0.028,-0.192609,-0.053690
4,YAL005C,-0.074,-0.055,-0.333106,-0.185473
5,YAL007C,-0.008,-0.018,0.037806,-0.004881
6,YAL008W,-0.034,-0.028,-0.108310,-0.053690


# Print out

In [27]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [1]:
# from IO.save_data_to_db3 import *

In [2]:
# save_data_to_db(data_all, paper_pmid)